In [4]:
import pandas as pd
from pysus.online_data.SIA import download
from pysus.utilities.readdbc import read_dbc
import os
import glob

# Configurações
output_dir = "/home/amcjr/pysus/processed/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Intervalo e condições do filtro
ano_inicial = 2023
ano_final = 2023
mes_inicial = 1
mes_final = 12
condicao = "PG"
tipo_financiamento = "06"

# Função para carregar e processar os dados
def processar_arquivos_sia(output_dir, ano_inicial, ano_final, mes_inicial, mes_final):
    # Construir o range de meses no formato 'YYYYMM'
    meses = [
        f"{ano}{str(mes).zfill(2)}"
        for ano in range(ano_inicial, ano_final + 1)
        for mes in range(mes_inicial, mes_final + 1)
    ]
    
    arquivos = glob.glob(os.path.join(output_dir, "*.dbc"))
    
    # Verifica se há arquivos para processar
    if not arquivos:
        raise ValueError("Nenhum arquivo DBC encontrado na pasta de saída.")
    
    dados = []
    
    for arquivo in arquivos:
        print(f"Processando arquivo: {arquivo}")
        # Lê o arquivo DBC e converte para DataFrame
        df = read_dbc(arquivo, encoding="iso-8859-1")
        dados.append(df)

    # Combinar todos os dados em um único DataFrame
    df_completo = pd.concat(dados, ignore_index=True)
    
    # Aplicar filtros
    df_filtrado = df_completo[
        (df_completo["CO_PA_MVM"].isin(meses)) &
        (df_completo["CO_PA_CONDIC"] == condicao) &
        (df_completo["CO_PA_TPFIN"] == tipo_financiamento)
    ]
    
    # Realizar o agrupamento e sumarização
    df_resumo = (
        df_filtrado.groupby(
            ["CO_PA_PROC_ID", "CO_PA_UFMUN", "CO_PA_MUNPCN", "CO_PA_DOCORIG"],
            as_index=False
        )
        .agg(
            NU_QUANTIDADE=("NU_PA_QTDAPR", "sum"),
            NU_VALOR=("NU_PA_VALAPR", "sum")
        )
    )
    
    return df_resumo

# Execução do processamento
try:
    df_resultado = processar_arquivos_sia(output_dir, ano_inicial, ano_final, mes_inicial, mes_final)
    print(df_resultado.head())  # Exibe uma prévia do DataFrame
    # Salvar em CSV ou outra saída
    df_resultado.to_csv(os.path.join(output_dir, "resumo_sia_2023.csv"), index=False)
    print("Resumo salvo com sucesso.")
except Exception as e:
    print(f"Erro durante o processamento: {e}")


ModuleNotFoundError: No module named 'pysus.utilities.readdbc'